<a href="https://colab.research.google.com/github/sanjayrawat2468/CapstoneProject2-EDA/blob/main/Capstone_Project_2_EDA_Type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - **NYC Taxi Trip Time Prediction**



##### **Project Type**    - EDA
##### **Contribution**    - Individual
##### **Team Member  -  Sanjay Rawat**


# **Project Summary**

## **To Explore various attributes and build a Predictive model that predicts the total trip duration of taxi trips in New York City.**



# **GitHub Link**

**https://github.com/sanjayrawat2468/CapstoneProject2-EDA/blob/main/Capstone_Project_2_EDA_Type.ipynb**

# **Problem Statement**


## **Your task is to build a model that predicts the total ride duration of taxi trips in New York City. Your primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and several other variables..**

# **Data Description**

## The dataset is based on the 2016 NYC Yellow Cab trip record data made available in Big Query on Google Cloud Platform. The data was originally published by the NYC Taxi and Limousine Commission (TLC). The data was sampled and cleaned for the purposes of this project. Based on individual trip attributes, you should predict the duration of each trip in the test set.


# **Data fields**
## **id -** A unique identifier for each trip.
## **vendor_id** - A code indicating the provider associated with the trip record.
## **pickup_datetime** - Date and time when the meter was engaged.
## **dropoff_datetime -** Date and time when the meter was disengaged.
## **passenger_count -** The number of passengers in the vehicle. (driver entered value)
## **pickup_longitude -** The longitude where the meter was engaged.
## **pickup_latitude -** The latitude where the meter was engaged.
## **dropoff_longitude -** The longitude where the meter was disengaged.
## **dropoff_latitude -** The latitude where the meter was disengaged.
## **store_and_fwd_flag -** This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip.
## **trip_duration -** Duration of the trip in seconds. (Target variable)



# ***Let's Begin !***

# **Import Libraries**

In [ ]:
# Importing Required Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.formula.api as sm
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings; warnings.simplefilter('ignore')


# **Mount Google Drive**


In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

# **Dataset Loading**

In [ ]:
# Load Dataset
data = pd.read_csv('/content/NYC Taxi Data.csv',on_bad_lines='skip')


# **Dataset First View**

In [ ]:
# Dataset First Look
data.head()

# **Dataset Rows & Columns count**

In [ ]:
# Dataset Rows & Columns count
print("Number of rows is: ", data.shape[0])
print("Number of columns is: ", data.shape[1])

# **Dataset Information**

In [ ]:
# Dataset Info
data.info

In [ ]:
data.describe()

# **Duplicate Values**

In [ ]:
# Dataset Duplicate Value Count
data.duplicated().sum()

# **Missing Values/Null Values**

In [ ]:
# Missing Values/Null Values Count
data.isnull().sum()

In [ ]:
# Visualizing the missing values
data.dropna(axis = 0,inplace = True)

In [ ]:
data.shape

# **What did you know about your dataset?**

**Originally data was containing the 106745 rows and 11 columns but after checking for null values i found that there are some null values i.e neglegible so I dropped them and now dataset having 106744 rows and 11 columns.**

# **Understanding Your Variables**

In [ ]:
# Dataset Columns
data.columns

# **Check Unique Values for each variable**

In [ ]:
# Check Unique Values for each variable.
print( 'We have %d unique id in our dataset ' %(data.id.nunique()))
print( 'We have %d unique vendor_id in our dataset ' %(data.vendor_id.nunique()))
print( 'We have %d unique pickup_datetime in our dataset ' %(data.pickup_datetime.nunique()))
print( 'We have %d unique dropoff_datetime in our dataset ' %(data.dropoff_datetime.nunique()))
print( 'We have %d unique passenger_count in our dataset ' %(data.passenger_count.nunique()))
print( 'We have %d unique pickup_longitude in our dataset ' %(data.pickup_longitude.nunique()))
print( 'We have %d unique pickup_latitude in our dataset ' %(data.pickup_latitude.nunique()))
print( 'We have %d unique dropoff_longitude in our dataset ' %(data.dropoff_longitude.nunique()))
print( 'We have %d unique dropoff_latitude in our dataset ' %(data.dropoff_latitude.nunique()))
print( 'We have %d unique store_and_fwd_flag in our dataset ' %(data.store_and_fwd_flag.nunique()))
print( 'We have %d unique trip_duration in our dataset ' %(data.trip_duration.nunique()))


In [ ]:
data.dtypes

# **Data Wrangling**

In [ ]:
# We have pickup_datetime, dropoff_datetime of the type 'object'. Convert it into type 'datetime'.
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])


In [ ]:
# let us extract or add some new features from existing ones
data['pickup_weekday']=data['pickup_datetime'].dt.day_name()
data['dropoff_weekday']=data['dropoff_datetime'].dt.day_name()
data['pickup_weekday_num']=data['pickup_datetime'].dt.weekday
data['pickup_hour']=data['pickup_datetime'].dt.hour
data['month']=data['pickup_datetime'].dt.month

**1** - pickup_weekday which will contain the name of the day on which the ride was taken.

**2** - pickup_weekday_num which will contain the day number instead of characters with Monday=0 and Sunday=6.

**3** - pickup_hour with an hour of the day in the 24-hour format.

**4** - pickup_month with month number with January=1 and December=12.

**Lets Extract some other features from the dataset as we have columns or features namely longitude and latitude.**

In [ ]:
# Importing tha required liberary for
from geopy.distance import great_circle

In [ ]:
# Lets create a function to get the distance feature from dataset
def cal_distance(pickup_lat,pickup_long,dropoff_lat,dropoff_long):
 
 start_coordinates=(pickup_lat,pickup_long)
 stop_coordinates=(dropoff_lat,dropoff_long)
 
 return great_circle(start_coordinates,stop_coordinates).km

In [ ]:
# Now applying the above function to dataset
data['distance'] = data.apply(lambda x: cal_distance(x['pickup_latitude'],x['pickup_longitude'],x['dropoff_latitude'],x['dropoff_longitude'] ), axis=1)

In [ ]:
# Calculating the speed with speed = distance/time formula
data['speed'] = (data.distance/(data.trip_duration/3600))

**Lets Define the different time segments acording to time as morning after noon and so on**

In [ ]:
# Lets create a function to define different time segments of a day.
def time_of_day(x):
    if x in range(6,12):
        return 'Morning'
    elif x in range(12,16):
        return 'Afternoon'
    elif x in range(16,22):
        return 'Evening'
    else:
        return 'Late night'

In [ ]:
# Applying the above function to the dataset
data['pickup_timeofday']=data['pickup_hour'].apply(time_of_day)

# **Exploratory Data Analysis**

## **Univariate Analysis**

In [ ]:
# Lets find the out the didtribution for the target variable i.e tripduration
plt.figure(figsize = (10,5))
sns.distplot(data['trip_duration'])
plt.xlabel('Trip Duration')
 
plt.show()

**From the above plot we can see that it is not normally ditributed or right skewed.**

In [ ]:
# Applying log tranformation to the target variable
plt.figure(figsize = (10,5))
sns.distplot(np.log10(data['trip_duration']))
plt.xlabel('Trip Duration')
plt.show()

**Now the target variable is normally ditributed.**